# Численное интегрирование

## Введение

Данная работа посвящена оценке точности различных методов численного 
интегрирования. Для примера возьмем функцию, заданную таблично из VII.9.5 (a).

## Методология 

Сравнивать будем следующие методы:
1. метод прямоугольников 
2. метод трапеций
3. метод симпсона
4. метод Гаусса

## Исследование

In [19]:
import numpy as np

In [20]:
data = {
  "x": [0.25 * i for i in range(9)],
  "f(x)": [
    1.000000, 0.989616, 0.958851, 0.908852, 0.841471, 0.759188, 0.664997,
    0.562278, 0.454649
  ]
}

x = data["x"]
y = data["f(x)"]

In [21]:
def check_xy(x, y):
  x = np.asarray(x, dtype=float)
  y = np.asarray(y, dtype=float)
  if x.ndim != 1 or y.ndim != 1:
    raise ValueError("x и y должны быть 1-мерными массивами")
  if x.size != y.size:
    raise ValueError("x и y должны иметь одинаковую длину")
  if np.any(np.diff(x) <= 0):
    raise ValueError("x должен быть строго возрастающим")
  return x, y


def interp_linear(x, y, xs):
  """Линейная интерполяция значений y в точках xs. Возвращает np.array.
      (аналог np.interp, но с поддержкой массивов и выхода за пределы - экстраполяция)
  """
  x = np.asarray(x)
  y = np.asarray(y)
  xs = np.asarray(xs)
  # np.interp не поддерживает массивы x не-возрастающие; мы уже проверяли.
  return np.interp(xs, x, y, left=None, right=None)


def composite_rectangle(x, y, kind='midpoint'):
  """
  Композитный метод прямоугольников по табличным данным.
  kind: 'left', 'right', 'midpoint'
  Для midpoint значения в серединах отрезков берутся линейной интерполяцией.
  Возвращает приближение интеграла на [x0, xn].
  """
  x, y = check_xy(x, y)
  h = np.diff(x)
  if kind == 'left':
    vals = y[:-1]
  elif kind == 'right':
    vals = y[1:]
  elif kind == 'midpoint':
    mids = (x[:-1] + x[1:]) / 2.0
    vals = interp_linear(x, y, mids)
  else:
    raise ValueError("kind должен быть 'left', 'right' или 'midpoint'")
  return np.sum(vals * h)


def trapezoidal(x, y):
  """Составной метод трапеций для табличных точек (произвольные шаги)."""
  x, y = check_xy(x, y)
  h = np.diff(x)
  return np.sum((y[:-1] + y[1:]) * 0.5 * h)


def integrate_quadratic_on_interval(x0, y0, x1, y1, x2, y2):
  """
  На вход три точки (xi, yi). Строим квадратичный многочлен p(x)
  через эти три точки (решаем Вандермондову систему) и интегрируем p(x) на [x0, x2].
  Возвращаем точное значение интеграла полинома.
  """
  # Построим систему для коэффициентов a + b*x + c*x^2
  A = np.array([[1, x0, x0**2], [1, x1, x1**2], [1, x2, x2**2]], dtype=float)
  rhs = np.array([y0, y1, y2], dtype=float)
  a, b, c = np.linalg.solve(A, rhs)
  # Интеграл от a + b x + c x^2:
  I = a * (x2 - x0) + b * 0.5 * (x2**2 - x0**2) + c * (1.0 / 3.0) * (x2**3 -
                                                                     x0**3)
  return I


def simpson(x, y):
  """
  Композитный метод Симпсона по табличным данным.
  Работает по парам интервалов (каждая итерация использует 3 смежные точки).
  Если число интервалов нечетное, то последний интервал приближён методом трапеций.
  Поддерживает неравномерную сетку, т.к. интеграл на каждой паре точек вычисляется точно
  через квадратичную аппроксимацию (интеграл интерполирующего многочлена).
  """
  x, y = check_xy(x, y)
  n = x.size
  if n < 3:
    raise ValueError("Нужно как минимум 3 точки для Симпсона")
  total = 0.0
  i = 0
  intervals = n - 1
  while i + 2 < n:
    # используем точки i, i+1, i+2
    total += integrate_quadratic_on_interval(x[i], y[i], x[i + 1], y[i + 1],
                                             x[i + 2], y[i + 2])
    i += 2
  if i + 1 < n:
    # остался один интервал [x[i], x[i+1]] - применим трапецию
    total += 0.5 * (y[i] + y[i + 1]) * (x[i + 1] - x[i])
  return total


gauss_coeffs = {
  1: (np.array([0.0]), np.array([2.0])),
  2: (np.array([-0.5773503, 0.5773503]), np.array([1.0, 1.0])),
  3: (np.array([-0.7745967, 0.0,
                0.7745967]), np.array([0.5555556, 0.8888889, 0.5555556])),
  4: (np.array([-0.8611363, -0.3399810, 0.3399810, 0.8611363]),
      np.array([0.3478548, 0.6521451, 0.6521451, 0.3478548])),
  5: (np.array([-0.9061798, -0.5384693, 0.0, 0.5384693, 0.9061798]),
      np.array([0.4786287, 0.2369269, 0.5688888, 0.2369269, 0.4786287])),
  6: (np.array(
    [-0.9324700, -0.6612094, -0.2386142, 0.2386142, 0.6612094, 0.9324700]),
      np.array(
        [0.1713245, 0.3607616, 0.4679140, 0.4679140, 0.3607616, 0.1713245]))
}


def gauss_legendre(x, y, n=6):
  """
  Правило Гаусса-Лежандра с n узлами (n=1,2,3,4,5,6) на всём отрезке [x0, xn].
  Для вычисления f в узлах используется линейная интерполяция по табличным данным.
  Возвращает приближение интеграла.
  """
  if n not in gauss_coeffs:
    raise ValueError("Поддерживаются только n=1,2,3,4,5,6")
  x, y = check_xy(x, y)
  a, b = x[0], x[-1]
  nodes, weights = gauss_coeffs[n]
  # отображаем узлы из [-1,1] в [a,b]:  t -> (b+a)/2 + (b-a)/2 * t
  mapped = 0.5 * (b + a) + 0.5 * (b - a) * nodes
  fvals = interp_linear(x, y, mapped)
  integral = 0.5 * (b - a) * np.dot(weights, fvals)
  return integral

Докажем функциональную корректность:

In [22]:
import unittest
import scipy.integrate


class TestIntegration(unittest.TestCase):
  global x, y

  def test_trapezoid(self):
    self.assertAlmostEqual(scipy.integrate.trapezoid(y, x), trapezoidal(x, y))

  def test_simpson(self):
    self.assertAlmostEqual(scipy.integrate.simpson(y, x), simpson(x, y))


unittest.main(argv=[""], verbosity=2, exit=False)

test_simpson (__main__.TestIntegration.test_simpson) ... ok
test_trapezoid (__main__.TestIntegration.test_trapezoid) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [23]:
print(f"Метод прямоугольников: {composite_rectangle(x, y)}")
print(f"Метод трапеций: {trapezoidal(x, y)}")
print(f"Метод Симпсона (парабол): {simpson(x, y)}")
print(f"Метод Гаусса-Лежандра: {gauss_legendre(x, y)}")

Метод прямоугольников: 1.6031443749999998
Метод трапеций: 1.6031443749999998
Метод Симпсона (парабол): 1.6054185833333334
Метод Гаусса-Лежандра: 1.6036173216367557


Пример

In [25]:
xs = np.linspace(0.001, 1, 1000)
f1 = [1 / np.sqrt(x) for x in xs]
f2 = [(np.cos(x) - 1) / np.sqrt(x) for x in xs]

print(f"{simpson(xs, f1) + simpson(xs, f2)}")

1.745950900677275
